In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from functions import *
from models.transformer import *
from models.EGNN import *
from data_loader_one_step_UVS import load_test_data
from data_loader_one_step_UVS import load_train_data
device = "cpu" #"cuda" if torch.cuda.is_available() else "cpu"

pytorch verison = 2.1.2
2.1.2


In [2]:
# options for the model and training
dataset_path = "dataset"
result_path = "results"
#Parameters
path_outputs = "./outputs/"
lead = 1
delta_t =0.01

batch_size = 1000
lamda_reg =0.2
wavenum_init=0 #10
wavenum_init_ydir=0 #10

save_interval = 10 # how often to save the model
num_epochs = 5000
learning_rate = 1e-3 # 0.00001

#data load
#(50000, 64, 9)
x_tensor=np.load(os.path.join(dataset_path, "input.npy"))
s = np.shape(x_tensor)
print("train set shape : ", x_tensor.shape)
# x_tensor = np.reshape(x_tensor, (s[0]*s[1],s[2], s[3]))

d1,d2,d3 = np.shape(x_tensor)
x_tensor_norm, mean_list_x, std_list_x = normalize_md(x_tensor)
x_tensor_norm=np.reshape(x_tensor_norm, [d1,1,d2,d3])
x_tensor_norm=torch.from_numpy(x_tensor_norm)
y_tensor=np.load(os.path.join(dataset_path, "output.npy"))
s = np.shape(y_tensor)
#y_tensor = np.reshape(y_tensor, (s[0]*s[1],s[2], s[3]))
d1,d2,d3 = np.shape(y_tensor)
y_tensor_norm, mean_list_y, std_list_y = normalize_md(y_tensor)
y_tensor_norm=np.reshape(y_tensor_norm, [d1,1,d2,d3]) 
y_tensor_norm=torch.from_numpy(y_tensor_norm)

tr_x, te_x =  x_tensor_norm[:-1000], x_tensor_norm[-1000:]
tr_y, te_y =  y_tensor_norm[:-1000], y_tensor_norm[-1000:]
print(tr_x.shape, tr_y.shape, te_x.shape, te_y.shape) #[batch_size, channel, n_atom, atom_feature_size]
# Define beta schedule
T = 300
betas = linear_beta_schedule(timesteps=T)
# Pre-calculate different terms for closed form
alphas = 1. - betas
alphas_cumprod = torch.cumprod(alphas, axis=0)
alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1, 0), value=1.0)
sqrt_recip_alphas = torch.sqrt(1.0 / alphas)
sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
sqrt_one_minus_alphas_cumprod = torch.sqrt(1. - alphas_cumprod)
posterior_variance = betas * (1. - alphas_cumprod_prev) / (1. - alphas_cumprod)
psi_test_label_Tr = y_tensor_norm.detach().cpu().numpy()

train set shape :  (50000, 64, 9)
torch.Size([49000, 1, 64, 9]) torch.Size([49000, 1, 64, 9]) torch.Size([1000, 1, 64, 9]) torch.Size([1000, 1, 64, 9])


In [3]:
trainN=5000
for step in range(0,trainN-batch_size,batch_size):
    # get the inputs; data is a list of [inputs, labels]
    indices = np.random.permutation(np.arange(start=step, stop=step+batch_size))
    #print(indices)
    input_batch, label_batch = tr_x[indices,:,:,:], tr_y[indices,:,:,:]
    print(input_batch.shape, label_batch.shape)
    t = torch.randint(0, T, (batch_size,), device=device).long()
    print("t : ", t.shape)

torch.Size([1000, 1, 64, 9]) torch.Size([1000, 1, 64, 9])
t :  torch.Size([1000])
torch.Size([1000, 1, 64, 9]) torch.Size([1000, 1, 64, 9])
t :  torch.Size([1000])
torch.Size([1000, 1, 64, 9]) torch.Size([1000, 1, 64, 9])
t :  torch.Size([1000])
torch.Size([1000, 1, 64, 9]) torch.Size([1000, 1, 64, 9])
t :  torch.Size([1000])


In [4]:
# test_tensor = x_tensor[:10]
# def normalize_md(x):
#     #x: [5000,64,9]
#     #x_out = np.zeros_like(x)
#     d1,d2,d3 = np.shape(x)
#     mean_list = [np.average(x[:,:,i]) for i in range(d3)]
#     std_list = [np.std(x[:,:,i]) for i in range(d3)]
#     for i in range(d3):
#         for index_1 in range(d1):
#             for index_2 in range(d2):
#                  x[index_1,index_2,i] = (x[index_1,index_2,i]-mean_list[i])/(std_list[i])
#     #print(np.amax(x), np.amin(x), mean_list, std_list)
#     return x, mean_list, std_list
# print(test_tensor.shape)

In [5]:
def calc_distance(x:torch.Tensor):
    n_frames, n_atoms, n_features = x.shape
    assert n_features == 3,  "The last dimension should be 3"

    dist_mtx = torch.zeros((n_frames, n_atoms, n_atoms))
    x2 = torch.sum(torch.square(x), dim=-1)
    y2 = torch.sum(torch.square(x), dim=-1)
    xy = torch.matmul(x, x.transpose(-1, -2))
    dist_mtx = torch.sqrt(torch.maximum(x2[:, :, None] + y2[:, None, :] - 2 * xy, torch.tensor(1e-6)))

    return dist_mtx

In [6]:
model = EGNN(in_dim=64,
            out_dim=6,
            h_dim=128,
            num_layer=3,
            num_timesteps=300,
            update_coord=True,
            use_attention=True,
            num_head=4)
input_batch = input_batch.squeeze()
# [1000, 64, 9]
n_frames, n_atoms, n_features = input_batch.shape
x_coord, x_force_speed = torch.split(input_batch, [3, 6], dim=-1)
#print(input_batch)
print("input shape : ", x_coord.shape, x_force_speed.shape)

dist_mtx = calc_distance(x_coord)
mask = torch.ones((n_frames, n_atoms))
mask2d = dist_mtx < 1.0

new_feat, new_coord = model(atom_feat=x_force_speed.float(),
                            coord=x_coord.float(),
                            t=t.view(-1,1),
                            adj_mat=mask2d,
                            mask=mask,
                            mask2d=mask2d )

print(new_feat.shape, new_coord.shape)
out_pred = torch.cat([new_coord, new_feat], dim=-1)
print(out_pred.shape)


input shape :  torch.Size([1000, 64, 3]) torch.Size([1000, 64, 6])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64000x6 and 9x128)